In [5]:
"""
bdf_preprocess.py
Preprocesses SEEG .bdf files: loads data, sets channel types, applies filters,
performs bipolar and average re-referencing, saves outputs, and generates plots.
Author: Jay Lei
Date: Mar 3 2025
"""

import mne
import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob

def preprocess_seeg(file_path, bipolar_folder, average_folder, 
                    l_freq=0.5, h_freq=450, notch_freqs=np.arange(50, 451, 50)):
    """
    Preprocess a single SEEG .bdf file with filtering and re-referencing.
    
    Parameters:
    - file_path: Path to the .bdf file.
    - bipolar_folder: Directory to save bipolar re-referenced data.
    - average_folder: Directory to save average re-referenced data.
    - l_freq: Low frequency for bandpass filter (Hz).
    - h_freq: High frequency for bandpass filter (Hz).
    - notch_freqs: Frequencies for notch filter (Hz).
    """
    # Load the raw data
    raw = mne.io.read_raw_bdf(file_path, preload=True)
    
    # Set all channels as SEEG type
    raw.set_channel_types({ch: 'seeg' for ch in raw.ch_names})
    
    # Apply bandpass filter (0.5-450 Hz)
    raw.filter(l_freq=l_freq, h_freq=h_freq, method='fir')
    
    # Apply notch filter (50 Hz and harmonics)
    raw.notch_filter(freqs=notch_freqs, method='fir')
    
    # Bipolar re-referencing
    ch_names = raw.ch_names
    electrode1 = ch_names[0:4]  # Channels 1-4
    electrode2 = ch_names[4:8]  # Channels 5-8
    bipolar_anodes = electrode1[:-1] + electrode2[:-1]
    bipolar_cathodes = electrode1[1:] + electrode2[1:]
    raw_bipolar = mne.set_bipolar_reference(raw, anode=bipolar_anodes, 
                                            cathode=bipolar_cathodes, copy=True)
    
    # Average re-referencing
    raw_avg = raw.copy().set_eeg_reference('average')
    
    # Save preprocessed data
    base_name = os.path.splitext(os.path.basename(file_path))[0]
    bipolar_save_path = os.path.join(bipolar_folder, f"{base_name}_bipolar.fif")
    average_save_path = os.path.join(average_folder, f"{base_name}_average.fif")
    raw_bipolar.save(bipolar_save_path, overwrite=True)
    raw_avg.save(average_save_path, overwrite=True)
    
    # Visualize a 10-second segment of bipolar data
    if not os.path.exists('plots'):
        os.makedirs('plots')
    data, times = raw_bipolar[:, 0:10000]  # First 10 seconds (assuming 1000 Hz sampling rate)
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(times, data.T)
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Amplitude (µV)')
    ax.set_title(f'Bipolar Preprocessed Data: {base_name}')
    plt.tight_layout()
    plot_path = os.path.join('plots', f"{base_name}_bipolar.png")
    fig.savefig(plot_path)
    plt.close(fig)

if __name__ == "__main__":
    # Define directories
    data_folder = './data/wkj/'
    bipolar_folder = './preprocessed/bipolar'
    average_folder = './preprocessed/average'
    
    # Create output directories if they don’t exist
    os.makedirs(bipolar_folder, exist_ok=True)
    os.makedirs(average_folder, exist_ok=True)
    
    # Get list of .bdf and .BDF files
    bdf_files = glob(os.path.join(data_folder, '*.BDF')) + glob(os.path.join(data_folder, '*.bdf'))
    if not bdf_files:
        print("No .bdf/.BDF files found in ./data/")
    else:
        print(f"Found {len(bdf_files)} .bdf files to process.")
        for file_path in bdf_files:
            print(f"Processing {file_path}...")
            preprocess_seeg(file_path, bipolar_folder, average_folder)
        print("Preprocessing completed.")


Found 1209 .bdf files to process.
Processing ./data/wkj/24_08_15-D_15_25_03.BDF...
Extracting EDF parameters from /Users/jlei61/Desktop/PYs/MARBLE/data/wkj/24_08_15-D_15_25_03.BDF...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 10999  =      0.000 ...    10.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 4.5e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 450.00 Hz
- Upper transition bandwidth: 50.00 Hz (-6 dB cutoff frequency: 475.00 Hz)
- Filter length: 6601 samples (6.601 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
-------------

/var/folders/v2/8d08w89x7dzg4jz50r3j3cyc0000gn/T/ipykernel_92326/652730822.py:56: RuntimeWarning: This filename (/Users/jlei61/Desktop/PYs/MARBLE/preprocessed/bipolar/24_08_15-D_15_25_03_bipolar.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_bipolar.save(bipolar_save_path, overwrite=True)
/var/folders/v2/8d08w89x7dzg4jz50r3j3cyc0000gn/T/ipykernel_92326/652730822.py:57: RuntimeWarning: This filename (/Users/jlei61/Desktop/PYs/MARBLE/preprocessed/average/24_08_15-D_15_25_03_average.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_avg.save(average_save_path, overwrite=True)


Processing ./data/wkj/24_08_15-D_22_00_00.BDF...
Extracting EDF parameters from /Users/jlei61/Desktop/PYs/MARBLE/data/wkj/24_08_15-D_22_00_00.BDF...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 2194999  =      0.000 ...  2194.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 4.5e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 450.00 Hz
- Upper transition bandwidth: 50.00 Hz (-6 dB cutoff frequency: 475.00 Hz)
- Filter length: 6601 samples (6.601 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, z

/var/folders/v2/8d08w89x7dzg4jz50r3j3cyc0000gn/T/ipykernel_92326/652730822.py:56: RuntimeWarning: This filename (/Users/jlei61/Desktop/PYs/MARBLE/preprocessed/bipolar/24_08_15-D_22_00_00_bipolar.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_bipolar.save(bipolar_save_path, overwrite=True)


Closing /Users/jlei61/Desktop/PYs/MARBLE/preprocessed/bipolar/24_08_15-D_22_00_00_bipolar.fif
[done]
Overwriting existing file.
Writing /Users/jlei61/Desktop/PYs/MARBLE/preprocessed/average/24_08_15-D_22_00_00_average.fif
Closing /Users/jlei61/Desktop/PYs/MARBLE/preprocessed/average/24_08_15-D_22_00_00_average.fif
[done]


...

Processing ./data/wkj/2024_09_06_01_28_59.bdf...
Extracting EDF parameters from /Users/jlei61/Desktop/PYs/MARBLE/data/wkj/2024_09_06_01_28_59.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 150999  =      0.000 ...   150.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 4.5e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 450.00 Hz
- Upper transition bandwidth: 50.00 Hz (-6 dB cutoff frequency: 475.00 Hz)
- Filter length: 6601 samples (6.601 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, ze

/var/folders/v2/8d08w89x7dzg4jz50r3j3cyc0000gn/T/ipykernel_92326/652730822.py:56: RuntimeWarning: This filename (/Users/jlei61/Desktop/PYs/MARBLE/preprocessed/bipolar/2024_09_06_01_28_59_bipolar.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_bipolar.save(bipolar_save_path, overwrite=True)
/var/folders/v2/8d08w89x7dzg4jz50r3j3cyc0000gn/T/ipykernel_92326/652730822.py:57: RuntimeWarning: This filename (/Users/jlei61/Desktop/PYs/MARBLE/preprocessed/average/2024_09_06_01_28_59_average.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_avg.save(average_save_path, overwrite=True)


Processing ./data/wkj/2024_09_02_02_17_54.bdf...
Extracting EDF parameters from /Users/jlei61/Desktop/PYs/MARBLE/data/wkj/2024_09_02_02_17_54.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1621999  =      0.000 ...  1621.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 4.5e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 450.00 Hz
- Upper transition bandwidth: 50.00 Hz (-6 dB cutoff frequency: 475.00 Hz)
- Filter length: 6601 samples (6.601 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, z

/var/folders/v2/8d08w89x7dzg4jz50r3j3cyc0000gn/T/ipykernel_92326/652730822.py:56: RuntimeWarning: This filename (/Users/jlei61/Desktop/PYs/MARBLE/preprocessed/bipolar/2024_09_02_02_17_54_bipolar.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_bipolar.save(bipolar_save_path, overwrite=True)


OSError: [Errno 28] No space left on device